In [79]:
import pandas as pd
import numpy as np
import re
import json
from itertools import permutations
from itertools import combinations

pd.set_option('display.max_columns', None, 'display.max_rows', 100)

df = pd.read_csv('../../data/immoscout_cleaned_lat_lon_fixed_v9.csv', low_memory=False)

col_names = df.columns.array
col_names[0:2] = ['Index1', 'Index2']
df.columns = col_names

df_cleaned = pd.DataFrame()

In this part of the data analysis we focus primarily on the variables containing information about 'Living Space', 'Plot Area', 'Floor Space' and 'Floor'. For each one of them we find at least 6 columns containing relevant data (columns with corresponding names in German, French, Italian, English, one '..._merged' and one 'detail_responsive#...').  

For the living space there's also another column called 'Space extracted'.  

How the '_merged' column has been merged is unclear, therefore we cannot assume that this column contains all the data.  

# Living Space

To investigate, how we can get the full set of data for living space, let's first create a subset containing only the columns with data on the living space.

In [80]:
living_space = ['Living space', 'Wohnfläche', 'Surface habitable', 'Superficie abitabile', 'detail_responsive#surface_living', 'Living_space_merged', 'Space extracted']
df_living_space = df.loc[:, living_space]

The cumulative sum of the value counts per column can be checked against the 'merged'-count to see if any combination of value counts add up to the same number.

In [81]:
df_living_space[living_space[0:5]].count().cumsum() == df_living_space[living_space[5]].count()

Living space                        False
Wohnfläche                          False
Surface habitable                   False
Superficie abitabile                 True
detail_responsive#surface_living    False
dtype: bool

Here we see, that the cumulative sum of the values in columns 'Living space', 'Wohnfläche', 'Surface habitable', 'Superficie abitabile' adds up to the same amount as the 'Living_space_merged' value count.  

Let's see if the data contained in 'Living_space_merged' is actually the same data contained in columns 'Living space', 'Wohnfläche', 'Surface habitable' and 'Superficie abitabile'.  

To do so we first combine the respective columns into one so we can compare the columns.

In [82]:
df_living_space['living_space'] = df_living_space[living_space[0]].fillna('') + \
  (df_living_space[living_space[1]]).fillna('') + \
  (df_living_space[living_space[2]]).fillna('') + \
  (df_living_space[living_space[3]]).fillna('')

(df_living_space.loc[:, 'living_space'] == df_living_space.loc[:, living_space[5]].fillna('')).sum() == df.shape[0]

True

Counting the `True` values of the comparison of the two columns yields the same count as the column count of the dataset meaning that all rows of the two columns are identical.

Therefore we can confirm, that the '_merged'-column contains all information from the columns 'Living space', 'Wohnfläche', 'Surface habitable' and 'Superficie abitabile'.  

There are still two remaining columns with data about the living space: 'detail_responsive#surface_living' and 'Space extracted'.  

Let's check if there's any new data in 'detail_responsive#surface_living':

In [83]:
df_living_space['living_space'] = df_living_space['living_space'].replace('', np.NaN)

df_living_space[df_living_space['living_space'].isna() & df_living_space["detail_responsive#surface_living"].notna()]

,Living space,Wohnfläche,Surface habitable,Superficie abitabile,detail_responsive#surface_living,Living_space_merged,Space extracted,living_space
2,NaN,NaN,NaN,NaN,93 m²,NaN,93.0,NaN
39,NaN,NaN,NaN,NaN,97 m²,NaN,97.0,NaN
44,NaN,NaN,NaN,NaN,216 m²,NaN,216.0,NaN
170,NaN,NaN,NaN,NaN,70 m²,NaN,70.0,NaN
178,NaN,NaN,NaN,NaN,127 m²,NaN,127.0,NaN
...,...,...,...,...,...,...,...,...
13351,NaN,NaN,NaN,NaN,93 m²,NaN,93.0,NaN
13356,NaN,NaN,NaN,NaN,157 m²,NaN,157.0,NaN
13357,NaN,NaN,NaN,NaN,121 m²,NaN,121.0,NaN
13359,NaN,NaN,NaN,NaN,83 m²,NaN,83.0,NaN


Interesting. This output shows that not only 'detail_responsive#surface_living' but also 'Space extracted' contains more information on living space. In a first step we can merge the 'detail_responsive#surface_living' into 'living_space' and compare the counts.

In [84]:
df_living_space['living_space'] = df_living_space[living_space[5]].fillna('') + \
  (df_living_space[living_space[4]]).fillna('')

df_living_space['living_space'] = df_living_space['living_space'].replace('', np.nan)

df_living_space['living_space'].count(), df_living_space['Space extracted'].count()

(12304, 12308)

With all columns containing direct information about the living space except 'Space extracted' we get 12304 rows of data. 'Space extracted' has 12308, 4 more rows.

In [85]:
df_living_space.loc[df['Space extracted'].notna() & df_living_space['living_space'].isna(), :] 

,Living space,Wohnfläche,Surface habitable,Superficie abitabile,detail_responsive#surface_living,Living_space_merged,Space extracted,living_space
786,NaN,NaN,NaN,NaN,NaN,NaN,200.0,NaN
3380,NaN,NaN,NaN,NaN,NaN,NaN,210.0,NaN
3696,NaN,NaN,NaN,NaN,NaN,NaN,228.0,NaN
6506,NaN,NaN,NaN,NaN,NaN,NaN,200.0,NaN


Does the column 'Space extracted' contain the same information as the merged column plus 4 more rows? To check that, the data has to be parsed the same way. 

In [86]:
df_living_space['living_space'] = df_living_space.living_space.str.extract('(\d+)').astype(float)

df_living_space['living_space'].dtype == df_living_space['Space extracted'].dtype

True

Now with the same datatype, the data can be merged and compared once more.

In [87]:
df_living_space.loc[df_living_space['Space extracted'].notna() & df_living_space['living_space'].isna(), 'living_space'] = df_living_space.loc[df_living_space['Space extracted'].notna() & df_living_space['living_space'].isna(), 'Space extracted']
(df_living_space['Space extracted'].astype(float) == df_living_space['living_space']).count()

13378

And with this we can see that the column 'Space extracted' contained the most complete data of the investigated columns for living space. 4 more than the '..._merged' and 'detail_responsive#...' combined. 

In [88]:
df_living_space['living_space'] = df_living_space['Space extracted']
df_living_space.drop(['Living space', 'Wohnfläche', 'Surface habitable', 'Superficie abitabile', 'detail_responsive#surface_living', 'Living_space_merged', 'Space extracted'], axis=1, inplace=True)
df_living_space.living_space.describe()

count    12308.000000
mean       158.687195
std        120.631122
min          1.000000
25%        100.000000
50%        131.000000
75%        180.000000
max       4390.000000
Name: living_space, dtype: float64

Now the next question is, where did those 4 more rows come from?  
In the dataset we have many redundant columns but we discovered a pattern for the above described columns. They are all extracted from different forms (mobile, desktop, different languages) of the website immoscout24.ch and the column names 'description', 'detailed_description', 'table', 'details', 'details_structured' suggest that they may contain raw data. 

# Description

To get an idea of the information contained in the 'description' column, let's look at the `value_counts()`

In [89]:
df['description'].value_counts()

4.5 rooms, 153 m²«Duplex dans les combles avec 2 terrasses !»CHF 686,700.—Favourite                                                                               4
3.5 rooms, 98 m²«Belle promotion Minergie de 22 appartements au calme ! Du 2.5 pces au 4.5 pces !»CHF 495,000.—Favourite                                          4
5.5 rooms, 153 m²«####Les Vergers d Ollon#### à Ollon VD Magnifique Villa Mitoyenne avec un grand jardin d environ 1 000 m2 à vendre»CHF 1,260,000.—Favourite     4
5.5 rooms, 170 m²«NOUVELLE PROMOTION»CHF 1,795,000.—Favourite                                                                                                     4
2.5 rooms, 82 m²«Quartier Saint-Michel Appartement 2.5 pces au 3e»CHF 492,000.—Favourite                                                                          3
                                                                                                                                                                 ..
3.5 rooms, 101 m

Two things immediately stand out:
- The same description (and maybe more features) have been recorded multiple times for some observations
- There seems to be a distinctive pattern in the data contained in the 'description'-column.  

Since we cannot inspect every column manually, we've built a regex matching group to check, if the structure of the data is consistent in all observations.

In [90]:
df['description'].count()

13378

In [91]:
description_pattern = '\d+\.?\d? rooms, \d+ m²«.+»CHF [\d,]+\.'
df['description'].str.contains(description_pattern).sum()

11201

The column 'description' contains information for every observation in the dataset. 11201 of which follow the defined structure. What about the rest?

In [92]:
is_structured = df['description'].str.contains(description_pattern)
not_structured = is_structured[is_structured == False]
not_structured.count()

2177

In [93]:
df.iloc[not_structured.index]['description'].head(10)

7     4.5 rooms«Preishit! Grossräumige Wohnung mitte...
15    258 m²«Mehrgenerationenhaus mit grossem Garten...
21    4.5 rooms, 236 m²«Terrassenhaus mit malerische...
22    167 m²«EFH 6.5 davon 1 Zi-Studio (Büro / Praxi...
33    258 m²«Mehrgenerationenhaus mit grossem Garten...
35    4.5 rooms«Preishit! Grossräumige Wohnung mitte...
41    4.5 rooms, 150 m²«####Two in One#### mit Einli...
44    7.5 rooms, 216 m²«Verzauberndes Generationenha...
45    5 rooms, 104 m²«#### Top Einfamilienhaus einge...
52    5.5 rooms, 160 m²«FAMILIENGLÜCK MIT VIEL PLATZ...
Name: description, dtype: object

So not every row of 'description' contains complete information about the rooms, living space and price. Let's check how it compares to the information contained in other columns in the dataset. 

## Rooms

In [95]:
df_rooms = pd.DataFrame(df.loc[:, 'rooms'])
df_rooms['details_rooms'] = df['description'].str.extract('(\d+\.?\d?) rooms, ').astype(float)
df_rooms['details_rooms'].describe()

count    11998.000000
mean         5.149817
std          2.208763
min          1.500000
25%          3.500000
50%          4.500000
75%          5.500000
max         29.000000
Name: details_rooms, dtype: float64

11998 rows of rooms-data can be extracted from 'description'

## Living Space

In [96]:
df['description'].str.contains('\d+ m²«').sum()

12304

Interesting, here we see the same count as before, when we merged all columns except 'Space extracted'. Is it actually the same information?

In [97]:
df_living_space['extracted_ls'] = df['description'].str.extract('(\d+) m²«')
(df_living_space['extracted_ls'].astype(float) == df_living_space['living_space']).sum()

12304

In [98]:
df_living_space.loc[df_living_space['living_space'].notna() & df_living_space['extracted_ls'].isna(), :]

,living_space,extracted_ls
786,200.0,NaN
3380,210.0,NaN
3696,228.0,NaN
6506,200.0,NaN


In [99]:
df_living_space.drop('extracted_ls', axis=1, inplace=True)

Indeed! with this we have proven, that the column 'description' does not contain more information about the living space. But the question, where those 4 rows come from, remains unanswered for now.  

## Price

In [101]:
df_price = df.loc[:, ['price', 'price_cleaned']]
df_price['description_price'] = df['description'].str.extract('CHF ([\d.,]+)')
df_price['description_price'].count()

12359

We can extract 12359 rows of data containing information about the price. We'll keep that in mind and move forward.  

# Detailed Description

In [102]:
df['detailed_description'].value_counts()

Extract from the debt collection registerIn a few days by e-mail and by post at your home. Per invoice, for CHF 29.–Order the extract                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In this column we find, as the name suggests, the detailed description of the posting. This data does not follow any specific pattern and does not contain any reliable information on the features we are investigating in this notebook. Therefore we discard it.  
With nlp techniques applied it may become useful for fine tuning predictions though. 

# Table

In [103]:
df['table'].value_counts()

b <article class=####Box-cYFBPY hKrxoH####><h2 class=####Box-cYFBPY gZLPvm####>Main information</h2><table class=####DataTable__StyledTable-sc-1o2xig5-1 jbXaEC####><tbody><tr><td class=####DataTable__SimpleCell-sc-1o2xig5-2 DataTable__Cell-sc-1o2xig5-4 edrNfG dGBatU####>Municipality</td><td class=####DataTable__SimpleCell-sc-1o2xig5-2 DataTable__CellValue-sc-1o2xig5-3 edrNfG rJZBK####>Le Mouret</td></tr><tr><td class=####DataTable__SimpleCell-sc-1o2xig5-2 DataTable__Cell-sc-1o2xig5-4 edrNfG dGBatU####>Availability</td><td class=####DataTable__SimpleCell-sc-1o2xig5-2 DataTable__CellValue-sc-1o2xig5-3 edrNfG rJZBK####>On request</td></tr></tbody></table><hr class=####Divider-iprSaI bBhTLQ####/></article>                                                                                                                                                                                                                                                                                                 

In this column, we see information about the Municipality, Living space, Plot area, Availability, Floor and so on. Maybe this is where our information originates from?

## Living Space

In [104]:
df['table'].str.contains('Living space').sum()

11634

Countwise we're well below the data we've gathered so far for the living space. 
Let's also check if there are any values for rows where `living_space` is na:

In [105]:
df_living_space['table_ls'] = df['table'].str.extract('Living space.+####>(\d+) m').astype(float)
df_living_space.loc[df_living_space['living_space'].isna() & df_living_space['table_ls'].notna(), 'living_space'].count()

0

In [179]:
df_living_space.drop('table_ls', axis=1, inplace=True)

And this confirms, that we cannot extract any more information on the living space from the 'table' column. We'll now extract relevant variables for further investigation.

## Municipality

In [106]:
df['table'].str.contains('Municipality').sum() == df['Municipality'].count()

True

In [107]:
df['table_municipality'] = df['table'].str.extract('Municipality<\/td><td .+?####.+?####>(.+?)<')
df.loc[df['table_municipality'].fillna('') != df['Municipality'].fillna(''), ['table_municipality', 'Municipality']]

,table_municipality,Municipality
4,K&#252;ttigen,Küttigen
43,K&#252;ttigen,Küttigen
47,K&#252;ttigen,Küttigen
51,K&#252;ttigen,Küttigen
55,K&#252;ttigen,Küttigen
...,...,...
13291,F&#228;llanden,Fällanden
13293,"D&#252;bendorf, Kreis 7 (Zurich)","Dübendorf, Kreis 7 (Zurich)"
13295,F&#228;llanden,Fällanden
13297,"Z&#252;rich, Kreis 6 (Zurich)","Zürich, Kreis 6 (Zurich)"


It looks like 'table' contains the same amount of information about the municipality as 'Municipality' but with encoding issues, so no need to extract any data from here. 

## Plot Area

In [108]:
df['table'].str.contains('Plot area').sum() == df['Plot area'].count()

True

In [109]:
df_plot_area = df.loc[:, ['Plot area', 'Grundstücksfläche', 'Surface du terrain', 'Superficie del terreno', 'detail_responsive#surface_property', 'Plot_area_merged']]
df_plot_area['table_pa'] = df['table'].str.extract('Plot area<\/td><td .+?####.+?####>(.+?)<')
df_plot_area.loc[df_plot_area['table_pa'].fillna('') != df_plot_area['Plot area'].fillna(''), ['table_pa', 'Plot area']]

,table_pa,Plot area
1,222 m&#178;,222 m²
3,370 m&#178;,370 m²
5,1063 m&#178;,1063 m²
6,200 m&#178;,200 m²
9,2116 m&#178;,2116 m²
...,...,...
13370,575 m&#178;,575 m²
13374,284 m&#178;,284 m²
13375,160 m&#178;,160 m²
13376,853 m&#178;,853 m²


And the same goes for the `plot_area`

In [187]:
df_plot_area.drop('table_pa', axis=1, inplace=True)


## Availability

In [110]:
df['table'].str.contains('Availability').sum() == df['Availability'].count()

True

## Floor

In [111]:
df['table'].str.contains('Floor<').sum() == df['Floor'].count()

True

## Floor Space

In [112]:
df['table'].str.contains('Floor space<').sum() == df['Floor space'].count()

True

# Details

In [113]:
df['details'].value_counts()

4.5 rooms, 120 m²,     135
4.5 rooms, 110 m²,     132
4.5 rooms,             120
3.5 rooms, 100 m²,     111
4.5 rooms, 100 m²,     102
                      ... 
121 m²,                  1
6 rooms, 132 m²,         1
5.5 rooms, 340 m²,       1
393 m²,                  1
7.5 rooms, 385 m²,       1
Name: details, Length: 2741, dtype: int64

It looks like we've got another column with information about the rooms and living space (or some other space). Let's extract and compare it.  

## Living Space

In [115]:
df_living_space['details_ls'] = df['details'].str.extract(', (\d+) m').astype(float)
df_living_space['details_ls'].describe()

count    12431.000000
mean       176.496983
std        272.100450
min          1.000000
25%        100.000000
50%        132.000000
75%        183.000000
max       9681.000000
Name: details_ls, dtype: float64

This count is greater than the number before! But also the max value raises from 4390 to 9681. Upon manual inspection of the articles, it becomes clear that this number not always containes the `living_space` attribute, but sometimes also `plot_area` or `floor_space` and since there's no keyword to classify this value, we cannot use it. 

In [193]:
# df_living_space[df_living_space['details_ls'].notna() & df_living_space['living_space'].isna()]

,living_space,details_ls
53,NaN,95.0
88,NaN,284.0
141,NaN,114.0
182,NaN,259.0
186,NaN,259.0
...,...,...
13204,NaN,803.0
13207,NaN,1436.0
13278,NaN,345.0
13337,NaN,544.0


<!-- Okay, 378 rows with information, where 'living_space' is NA. But is this data even about living space? -->

In [194]:
# (df_living_space['details_ls'].fillna(0) == df_living_space['living_space'].fillna(0)).sum()

12744

In [195]:
# not_equal = df_living_space['details_ls'].fillna(0) != df_living_space['living_space'].fillna(0)
# df_living_space.loc[not_equal[not_equal == True].index, ['details_ls', 'living_space']]

,details_ls,living_space
15,NaN,258.0
22,NaN,167.0
33,NaN,258.0
53,95.0,NaN
57,NaN,167.0
...,...,...
13204,803.0,NaN
13207,1436.0,NaN
13278,345.0,NaN
13337,544.0,NaN


In [196]:
# df_living_space.loc[df_living_space['details_ls'].notna() & df_living_space['living_space'].isna(), 'living_space'] = df_living_space.loc[df_living_space['details_ls'].notna() & df_living_space['living_space'].isna(), 'details_ls']
# df_living_space['living_space'].count()

12686

In [197]:
# df_living_space.drop('details_ls', axis=1, inplace=True)

<!-- We could extract 378 more rows for the living space from 'details'.   -->

## Rooms

In [125]:
df_rooms['details_rooms'] = df['details'].str.extract('(\d+\.?\d?) rooms, ').astype(float)
df_rooms['details_rooms'].count() == df_rooms['rooms'].count()

True

In [126]:
df_rooms.loc[df_rooms['details_rooms'].notna() & df_rooms['rooms'].isna(), ['details_rooms', 'rooms']]

,details_rooms,rooms


We do not gather more rows of information on the room counts but what information is contained in each column?

In [129]:
df_rooms['details_rooms'].value_counts()

4.5     3481
3.5     2314
5.5     2186
6.5      918
2.5      849
7.5      362
5.0      343
6.0      333
7.0      298
8.0      228
4.0      222
8.5      195
3.0      173
9.0      135
10.0     125
2.0       85
11.0      85
1.5       85
12.0      66
9.5       64
10.5      42
14.0      28
13.0      23
12.5      23
11.5      21
15.0      20
20.0      13
13.5      12
14.5      11
18.0      11
16.0      11
17.0      11
21.0       5
23.0       4
19.0       4
15.5       3
17.5       3
25.0       2
16.5       2
21.5       1
22.0       1
29.0       1
Name: details_rooms, dtype: int64

In [128]:
df_rooms['details_rooms'].describe()

count    12799.000000
mean         5.157200
std          2.211891
min          1.500000
25%          3.500000
50%          4.500000
75%          5.500000
max         29.000000
Name: details_rooms, dtype: float64

In [130]:
df_rooms['rooms'].value_counts()

4.5     3481
3.5     2314
5.5     2186
6.5      918
2.5      849
7.5      362
5.0      343
6.0      333
7.0      298
8.0      228
4.0      222
8.5      195
3.0      173
9.0      135
10.0     125
2.0       85
11.0      85
1.5       85
12.0      66
9.5       64
10.5      42
14.0      28
13.0      23
12.5      23
11.5      21
15.0      20
20.0      13
13.5      12
14.5      11
18.0      11
16.0      11
17.0      11
21.0       5
23.0       4
19.0       4
15.5       3
17.5       3
25.0       2
16.5       2
21.5       1
22.0       1
29.0       1
Name: rooms, dtype: int64

In [123]:
df_rooms['rooms'].describe()

count    12799.000000
mean         5.157200
std          2.211891
min          1.500000
25%          3.500000
50%          4.500000
75%          5.500000
max         29.000000
Name: rooms, dtype: float64

It looks like the extracted information is more accurate and therefore we can drop the 'rooms' column.  

In [131]:
df_rooms['rooms'] = df_rooms.loc[:, 'details_rooms']
df_rooms.drop('details_rooms', axis=1, inplace=True)


In [132]:
df_rooms.describe()

,rooms
count,12799.000000
mean,5.157200
std,2.211891
min,1.500000
25%,3.500000
50%,4.500000
75%,5.500000
max,29.000000


# Details Structured

In [133]:
df['details_structured'].value_counts()

{'Municipality': 'Biberstein', 'Living space': '100 m²', 'Floor': '4. floor', 'Availability': 'On request', 'location': '5023 Biberstein, AG', 'description': '3.5 rooms, 100 m²«Luxuriöse Attika-Wohnung mit herrlicher Aussicht»CHF 1,150,000.—Favourite', 'detailed_description': 'DescriptionLuxuriöse Attika-Wohnung direkt an der Aare und angrenzend an die Landwirtschaftszone, mit unverbaubarer Weitsicht, grosszügiger Garage und Option auf ein zusätzliches Zimmer.Einzigartige Lage, top Aussicht und hochwertige Innenausstattung? Das alles bietet diese charmante Eigentumswohnung auf 100m2 im steuergünstigen Biberstein. Stadtnah gelegen und mit direktem Naturzugang sorgt sie für ein rundum angenehmes Wohngefühl.In der ganzen Wohnung sind hochwertige Materialien mit einem südländischen Touch verbaut. Der Boden ist mit einem Jurastein und die beiden Zimmer mit Holz versehen (mit Bodenheizung).In die Wohnung gelangt man über einen separaten Eingang, ein halbes Stockwerk vom gewachsenen Boden erh

Municipality, Living space, Plot area, Availability, location, description, detailed_description, url, table, Floor space, Floor

In [134]:
df['details_structured'].str.contains("'Municipality'").sum() == df['Municipality'].count()

True

In [135]:
df['details_structured'].str.contains("'Living space'").sum() == df['Living space'].count()

True

In [136]:
df['details_structured'].str.contains("'Plot area'").sum() == df['Plot area'].count()

True

In [137]:
df['details_structured'].str.contains("'Floor space'").sum() == df['Floor space'].count()

True

In [138]:
df['details_structured'].str.contains("'Availability'").sum() == df['Availability'].count()

True

In [139]:
df['details_structured'].str.contains("'location'").sum() == df['location'].count()

True

In [140]:
df['details_structured'].str.contains("'description'").sum() == df['description'].count()

True

In [141]:
df['details_structured'].str.contains("'detailed_description'").sum() == df['detailed_description'].count()

True

In [142]:
df['details_structured'].str.contains("'url'").sum() == df['url'].count()

True

In [143]:
df['details_structured'].str.contains("'table'").sum() == df['table'].count()

True

In [144]:
df['details_structured'].str.contains("'Floor'").sum() == df['Floor'].count()

True

## Plot Area

In [145]:
df_plot_area

,Plot area,Grundstücksfläche,Surface du terrain,Superficie del terreno,detail_responsive#surface_property,Plot_area_merged,table_pa
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,222 m²,NaN,NaN,NaN,NaN,222 m²,222 m&#178;
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,370 m²,NaN,NaN,NaN,NaN,370 m²,370 m&#178;
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
13373,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13374,284 m²,NaN,NaN,NaN,NaN,284 m²,284 m&#178;
13375,160 m²,NaN,NaN,NaN,NaN,160 m²,160 m&#178;
13376,853 m²,NaN,NaN,NaN,NaN,853 m²,853 m&#178;


In [146]:
plot_area = ['Plot area', 'Grundstücksfläche', 'Surface du terrain', 'Superficie del terreno', 'detail_responsive#surface_property', 'Plot_area_merged']

df_plot_area[plot_area[0:5]].count().cumsum() == df_plot_area[plot_area[5]].count()

Plot area                             False
Grundstücksfläche                     False
Surface du terrain                    False
Superficie del terreno                 True
detail_responsive#surface_property    False
dtype: bool

In [147]:
df_plot_area['plot_area'] = df_plot_area[plot_area[0]].fillna('') + \
  (df_plot_area[plot_area[1]]).fillna('') + \
  (df_plot_area[plot_area[2]]).fillna('') + \
  (df_plot_area[plot_area[3]]).fillna('')

(df_plot_area['plot_area'] == df_plot_area[plot_area[5]].fillna('')).sum()

13378

In [148]:
df_plot_area['plot_area'] = df_plot_area[plot_area[5]].fillna('') + \
  (df_plot_area[plot_area[4]]).fillna('')

df_plot_area['plot_area'] = df_plot_area['plot_area'].replace('', np.nan)

df_plot_area['plot_area'].count()

4953

In [149]:
df_plot_area['plot_area'] = df_plot_area.plot_area.str.extract('(\d+)').astype(float)
df_plot_area.drop(['Plot area', 'Grundstücksfläche', 'Surface du terrain', 'Superficie del terreno', 'detail_responsive#surface_property', 'Plot_area_merged'], axis=1, inplace=True)
df_plot_area.describe()

,plot_area
count,4953.000000
mean,934.845548
std,1174.532146
min,10.000000
25%,329.000000
50%,600.000000
75%,1024.000000
max,9681.000000


## Floor Space

In [150]:
floor_space = ['Floor space', 'Nutzfläche', 'Surface utile', 'Superficie utile', 'detail_responsive#surface_usable', 'Floor_space_merged']
df_floor_space = df.loc[:, floor_space]

df_floor_space[floor_space[0:5]].count().cumsum() == df_floor_space[floor_space[5]].count()

Floor space                         False
Nutzfläche                          False
Surface utile                       False
Superficie utile                     True
detail_responsive#surface_usable    False
dtype: bool

In [151]:
df_floor_space['floor_space'] = df_floor_space[floor_space[0]].fillna('') + \
  (df_floor_space[floor_space[1]]).fillna('') + \
  (df_floor_space[floor_space[2]]).fillna('') + \
  (df_floor_space[floor_space[3]]).fillna('')

(df_floor_space['floor_space'] == df_floor_space[floor_space[5]].fillna('')).sum()

13378

In [152]:
# df_floor_space['floor_space'] = df_floor_space[floor_space[0]].fillna('') + \
#   (df_floor_space[floor_space[1]]).fillna('') + \
#   (df_floor_space[floor_space[2]]).fillna('') + \
#   (df_floor_space[floor_space[3]]).fillna('')

df_floor_space[df_floor_space['floor_space'] != '']['floor_space'].count()

2842

In [153]:
df_floor_space.Floor_space_merged.str.extract('(\d+)').astype(float).describe()

,0
count,2842.000000
mean,209.717804
std,273.712382
min,4.000000
25%,107.000000
50%,155.000000
75%,243.750000
max,7549.000000


In [154]:
df_floor_space['floor_space'] = df_floor_space[floor_space[5]].fillna('') + \
  (df_floor_space[floor_space[4]]).fillna('')

df_floor_space['floor_space'] = df_floor_space['floor_space'].replace('', np.nan)

df_floor_space['floor_space'].count()

2953

In [155]:
df_floor_space['floor_space'] = df_floor_space.floor_space.str.extract('(\d+)').astype(float)
df_floor_space.drop(['Floor space', 'Nutzfläche', 'Surface utile', 'Superficie utile', 'detail_responsive#surface_usable', 'Floor_space_merged'], axis=1, inplace=True)
df_floor_space.floor_space.describe()

count    2953.000000
mean      208.836776
std       269.952305
min         4.000000
25%       107.000000
50%       154.000000
75%       243.000000
max      7549.000000
Name: floor_space, dtype: float64

## Floor

In [156]:
floor = ['Floor', 'Stockwerk', 'Étage', 'Piano', 'detail_responsive#floor', 'Floor_merged']
df_floor = df.loc[:, floor]

df_floor[floor[0:5]].count().cumsum() == df_floor[floor[5]].count()

Floor                      False
Stockwerk                  False
Étage                      False
Piano                       True
detail_responsive#floor    False
dtype: bool

In [244]:
df_floor['floor'] = df_floor[floor[0]].fillna('') + \
  (df_floor[floor[1]]).fillna('') + \
  (df_floor[floor[2]]).fillna('') + \
  (df_floor[floor[3]]).fillna('')

(df_floor['floor'] == df_floor[floor[5]].fillna('')).sum()

13378

In [245]:
# df_floor['floor'] = df_floor[floor[0]].fillna('') + \
#   (df_floor[floor[1]]).fillna('') + \
#   (df_floor[floor[2]]).fillna('') + \
#   (df_floor[floor[3]]).fillna('')

df_floor[df_floor['floor'] != '']['floor'].count()

5414

In [246]:
df_floor['floor'] = df_floor[floor[5]].fillna('') + \
  (df_floor[floor[4]]).fillna('')

df_floor['floor'] = df_floor['floor'].replace('', np.nan)

df_floor['floor'].count()

5620

In [248]:
df_floor.drop(['Floor', 'Stockwerk', 'Étage', 'Piano', 'detail_responsive#floor', 'Floor_merged'], axis=1, inplace=True)
def parse_floor(x):
  if x != x:
    return np.nan
  elif x == 'Ground floor':
    return 0
  elif re.search('\. floor', x):
    return re.search('\d+', x).group()
  elif re.search('Basement', x):
    return '-' + re.search('\d+', x).group()


df_floor['floor'] = df_floor['floor'].apply(parse_floor).astype(float)
df_floor['floor'].unique()
df_floor

,floor
0,4. floor
1,NaN
2,2. floor
3,NaN
4,Ground floor
...,...
13373,NaN
13374,NaN
13375,NaN
13376,NaN


# Availability

In [249]:
availability = ['Availability', 'Verfügbarkeit', 'Disponibilité', 'Disponibilità', 'detail_responsive#available_from',  'Availability_merged']
df_availability = df.loc[:, availability]

df_availability.iloc[:, 0:5].count().cumsum() == df_availability.iloc[:, 5].count()

Availability                        False
Verfügbarkeit                       False
Disponibilité                       False
Disponibilità                        True
detail_responsive#available_from    False
dtype: bool

In [250]:
df_availability['availability'] = df_availability[availability[0]].fillna('') + \
  (df_availability[availability[1]]).fillna('') + \
  (df_availability[availability[2]]).fillna('') + \
  (df_availability[availability[3]]).fillna('')

(df_availability['availability'] == df_availability[availability[5]].fillna('')).sum()

13378

In [251]:
df_availability['availability'] = df_availability[availability[5]].fillna('') + \
  (df_availability[availability[4]]).fillna('')

df_availability['availability'] = df_availability['availability'].replace('', np.nan)

df_availability['availability'].count()

13378

In [253]:
df_availability.drop(['Availability', 'Verfügbarkeit', 'Disponibilité', 'Disponibilità', 'detail_responsive#available_from', 'Availability_merged'], axis=1, inplace=True)
df_availability

,availability
0,On request
1,On request
2,Immediately
3,On request
4,On request
...,...
13373,On request
13374,On request
13375,On request
13376,Immediately


# Gross return

In [213]:
df['Gross return'].unique()

array([nan, '0.00 %', '4.5 %'], dtype=object)

In [214]:
df['Gross return'].count()

6

This column does not contain a whole lot of information, therefore we will not consider it for our analysis. 

# Price

In [265]:
df_price

,price,price_cleaned,description_price
0,"CHF 1,150,000.—",1150000.0,"1,150,000."
1,"CHF 1,420,000.—",1420000.0,"1,420,000."
2,"CHF 720,000.—",720000.0,"720,000."
3,"CHF 1,430,000.—",1430000.0,"1,430,000."
4,"CHF 995,000.—",995000.0,"995,000."
...,...,...,...
13373,"CHF 1,101,000.—",1101000.0,"1,101,000."
13374,"CHF 1,750,000.—",1750000.0,"1,750,000."
13375,"CHF 1,415,000.—",1415000.0,"1,415,000."
13376,"CHF 1,465,000.—",1465000.0,"1,465,000."


In [266]:
df_price['price'] = df_price['price'].str.extract('CHF ([\d,]+)').replace(',', '', regex=True).astype(float)
df_price['price'].describe()

0        1150000.0
1        1420000.0
2         720000.0
3        1430000.0
4         995000.0
           ...    
13373    1101000.0
13374    1750000.0
13375    1415000.0
13376    1465000.0
13377    2750000.0
Name: price, Length: 13378, dtype: float64

In [267]:
df_price['description_price'] = df_price['description_price'].str.extract('([\d,]+)').replace(',', '', regex=True).astype(float)
df_price['description_price'].describe()

0        1150000.0
1        1420000.0
2         720000.0
3        1430000.0
4         995000.0
           ...    
13373    1101000.0
13374    1750000.0
13375    1415000.0
13376    1465000.0
13377    2750000.0
Name: description_price, Length: 13378, dtype: float64

In [271]:
(df_price['description_price'] == df_price['price']).count()

13378

In [272]:
(df_price['price'] == df_price['price_cleaned']).count()

13378

In [273]:
df_price.drop(['price_cleaned', 'description_price'], axis=1, inplace=True)

# Function

In [286]:
# def clean_integers(df):
#   '''Cleans the following columns in a dataframe and removes unnecessary columns:
#   - Living Space
#   - Plot Area
#   - Floor Space
#   - Floor
#   - Rooms
#   - Price
#   - Availability

#   Args:
#       df (pandas DataFrame): The .csv as dataframe
#   '''
#   def parse_floor(x):
#     if x != x:
#       return np.nan
#     elif x == 'Ground floor':
#       return 0
#     elif re.search('\. floor', x):
#       return re.search('\d+', x).group()
#     elif re.search('Basement', x):
#       return '-' + re.search('\d+', x).group()

#   col_names = df.columns.array
#   col_names[0:2] = ['Index1', 'Index2']
#   df.columns = col_names

#   df_clean = pd.DataFrame()  

#   # Merge columns
#   ## Living Space
#   df['living_space'] = df['Space extracted']
#   df['details_ls'] = df['details'].str.extract(', (\d+) m').astype(float)
#   df.loc[df['details_ls'].notna() & df['living_space'].isna(), 'living_space'] = df.loc[df['details_ls'].notna() & df['living_space'].isna(), 'details_ls']
#   df_clean['living_space'] = df['living_space'].astype(float)
#   ## Rooms
#   df_clean['rooms'] = df['details'].str.extract('(\d+\.?\d?) rooms, ').astype(float)
#   ## Plot Area
#   df_clean['plot_area'] = df['Plot_area_merged'].fillna('') + \
#     df['detail_responsive#surface_property'].fillna('')
#   ## Floor Space
#   df_clean['floor_space'] = df['Floor_space_merged'].fillna('') + \
#     df['detail_responsive#surface_usable'].fillna('')
#   ## FLoor
#   df_clean['floor'] = df['Floor_merged'].fillna('') + \
#     df['detail_responsive#floor'].fillna('')
#   ## Availability
#   df_clean['availability'] = df['Availability_merged'].fillna('') + \
#     df['detail_responsive#available_from'].fillna('')
#   ## Price
#   df_clean['price'] = df['price_cleaned']

#   # Parsing
#   df_clean['plot_area'] = df_clean['plot_area'].replace('', np.nan).str.extract('(\d+)').astype(float)
#   df_clean['floor_space'] = df_clean['floor_space'].replace('', np.nan).str.extract('(\d+)').astype(float)
#   df_clean['floor'] = df_clean['floor'].replace('', np.nan).apply(parse_floor).astype(float)
#   df_clean['availability'] = df_clean['availability'].replace('', np.nan)
#   df_clean = df_clean.join(df.loc[:, 'ForestDensityL':'type'])
#   df_clean.drop('price_cleaned', axis=1, inplace=True)

#   return df_clean  

In [287]:
# df_cleaned = clean_integers(pd.read_csv('../data/immoscout_cleaned_lat_lon_fixed_v9.csv', low_memory=False))
# df_cleaned.head()

,living_space,rooms,plot_area,floor_space,floor,availability,price,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Locality,Longitude,NoisePollutionRailwayL,NoisePollutionRailwayM,NoisePollutionRailwayS,NoisePollutionRoadL,NoisePollutionRoadM,NoisePollutionRoadS,PopulationDensityL,PopulationDensityM,PopulationDensityS,RiversAndLakesL,RiversAndLakesM,RiversAndLakesS,WorkplaceDensityL,WorkplaceDensityM,WorkplaceDensityS,Zip,distanceToTrainStation,gde_area_agriculture_percentage,gde_area_forest_percentage,gde_area_nonproductive_percentage,gde_area_settlement_percentage,gde_average_house_hold,gde_empty_apartments,gde_foreigners_percentage,gde_new_homes_per_1000,gde_politics_bdp,gde_politics_cvp,gde_politics_evp,gde_politics_fdp,gde_politics_glp,gde_politics_gps,gde_politics_pda,gde_politics_rights,gde_politics_sp,gde_politics_svp,gde_pop_per_km2,gde_population,gde_private_apartments,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,type
0,100.0,3.5,NaN,NaN,4.0,On request,1150000.0,0.511176,0.286451,0.090908,47.415927,Biberstein,8.085840,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,5023,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,penthouse
1,156.0,4.5,222.0,242.0,NaN,On request,1420000.0,0.511176,0.286451,0.090908,47.415927,Biberstein,8.085840,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,5023,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,terrace-house
2,93.0,2.5,NaN,NaN,2.0,Immediately,720000.0,0.163362,0.095877,0.001911,47.397416,Aarau,8.043150,0.0,0.0,0.0,0.334957,0.381257,0.297575,0.325887,0.393783,0.635194,0.154274,0.188229,0.000000,0.172646,0.163850,0.165830,5000,0.909587,11.354420,32.197891,7.137064,49.310624,2.01,2.023799,21.358623,3.814582,3.633134,5.324421,3.782202,18.089552,7.899807,8.851305,NaN,0.735032,26.515854,22.662290,1704.700162,21036.0,10149.0,3.549010,6.05,37.0,3092.0,30364.0,33493.0,penthouse
3,154.0,4.5,370.0,257.0,NaN,On request,1430000.0,0.511176,0.286451,0.090908,47.415927,Biberstein,8.085840,0.0,0.0,0.0,0.058298,0.067048,0.103850,0.092914,0.209530,0.366674,0.082170,0.001811,0.011871,0.030169,0.052120,0.098951,5023,3.038467,30.676329,51.449275,4.589372,13.285024,2.23,1.994681,9.255663,4.739336,5.873715,4.579662,3.359031,18.355360,6.057269,7.066814,NaN,0.220264,20.392805,30.809471,376.829268,1545.0,686.0,2.234259,5.89,14.0,9.0,308.0,331.0,detached-house
4,142.0,4.5,NaN,NaN,0.0,On request,995000.0,0.333865,0.279276,0.145835,47.404870,Rombach,8.052781,0.0,0.0,0.0,0.133498,0.132933,0.235917,0.190986,0.136984,0.204549,0.109586,0.141473,0.091805,0.046950,0.038008,0.055509,5022,1.460245,33.137090,49.705635,1.177460,15.979815,2.28,0.691563,15.901990,1.160862,5.217740,5.728026,5.006679,19.158429,6.502805,7.477959,NaN,0.892332,20.459524,27.590168,511.008403,6081.0,2638.0,1.708126,6.30,65.0,349.0,941.0,1355.0,flat
